# Seruminstituttet og Sundhedsstyrelses prognose af 22. marts 2020.


Denne *notebook* baserer sig på modellen, der er skitseret i notatet,
*Håndtering af COVID-19: Prognose og kapacitet i Danmark for intensiv
terapi* [ITA_COVID_19_](ITA_COVID_19_220320.pdf)

Notatet er udarbejdet sammen med RUC uden nærmere angivelse, måske 
[professor Lone Simonsen](https://forskning.ruc.dk/da/persons/lonesimo), der forsker i modellering 
af pandemier.

Lad os komme i gang. Først har vi brug for nogle databehandlingspakker:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.integrate import odeint

## SIR-modellen
[SIR-modellen](https://da.wikipedia.org/wiki/SIR-modellen) er en simpel infektionsmodel.

S, I, og R står for henholdsvis *Susceptible* (hvor man kan få sygdommen, som ikke har haft den endnu), 
*Infectuous* (hvor mange har den nu og kan smitte andre), og
*Recovered* (dem der ikke længere har sygdommen, både de raske og de døde).

Udviklingen over tid beskrives med nogle simple differentialligninger.

Modellen er også vist med kodeeksempler i [SciPython-bogen](https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/).
Disse danner udgangspunkt for nedenstående.


Notatet skifter lidt mellem en befolkning på 5,8 mio. og 5,7 mio. personer i Danmark.
Vi regner med 5,8.

Se Danmarks Statistiks [Folketal](https://www.dst.dk/da/Statistik/emner/befolkning-og-valg/befolkning-og-befolkningsfremskrivning/folketal)
 

In [ ]:
# Total befolkning, N.
N = 5_800_000

In [ ]:
# Udgangspunktet, antal personer i I hhv. R tilstandene, I0 and R0.
I0, R0 = 1, 0

# Resten er *susceptible* ved start, S0.
S0 = N - I0 - R0

# Beta beskriver hvor smitsom sygdommen er
# Gamma beskriver hvor længe de er smitsomme i middel (1/d dage)

# Notatet antager folk kan smitte i 7 dage
gamma = 1./7  

# Notatet antager, at syge smitter 2,6 personer (forholdet mellem beta og gamma)
beta = 2.6*gamma

In [ ]:
# Tid i dage (notatet siger 12 uger, men figur 2 er regnet på 15 uger)
UGER = 15
t = np.linspace(0, UGER*7, UGER*7)

In [ ]:
# SIR modellens differentialligninger
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# Starttilstandsvektor
y0 = S0, I0, R0
# Integrer SIR-modellen equations over the tidsaksen, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

Sådan ser totalerne ud 

In [ ]:
# Plot the data on three separate curves for S(t), I(t) and R(t)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(t, S/1_000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(t, I/1_000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(t, R/1_000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Time /days')
ax.set_ylabel('Number (1000s)')
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()

Lad os prøve at reproducere notatets figur 2:

TODO

In [ ]:
delta_S = S[1:] - S[:-1]
nye_smittede = -delta_S
max_nye_smittede = np.max(nye_smittede)
t_nye_smittede = t[1:]

In [ ]:
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(t_nye_smittede, nye_smittede, 'r', alpha=0.5, lw=2, label='Nye smittede')
plt.suptitle('Figur 2 fra notatet')
plt.axhline(y=max_nye_smittede, color='grey', linestyle='--', label="max")
plt.show()

 
## Empiriske data
Data kan hentes fra Sundhedsstyrelsen, [Tal og overvågning](https://www.sst.dk/da/corona/tal-og-overvaagning).